## Scan with schedule file

Testing ways to simulate scanning from the Atacama desert using a schedule file

In [ ]:
import os, pickle, time
import numpy as np
import healpy as hp
from beamconv import ScanStrategy, tools
import matplotlib.pyplot as plt
import qpoint as qp
import seaborn as sns
sns.set()
%matplotlib inline

In [ ]:
def get_cls(fname='../ancillary/wmap7_r0p03_lensed_uK_ext.txt'):
    cls = np.loadtxt(fname, unpack=True) # Cl in uK^2
    return cls[0], cls[1:]

In [ ]:
# Load up alm                                                                            
lmax = 700
ell, cls = get_cls()
np.random.seed(39)
alm = hp.synalm(cls, lmax=lmax, new=True, verbose=True) # uK  

In [ ]:
# init scan strategy and instrument                                                      

nsamp      = 10000
mmax       = 5
fwhm       = 30.
scan_speed = 2.8

beam_opts = dict(lmax=800,
                 fwhm=fwhm,       # Gaussian co-pol beam, so only specify FWHM (arcmin)
                 btype='Gaussian')

soscan = ScanStrategy(duration=nsamp,
                  sample_rate=10.21,
                  location='atacama')

soscan.create_focal_plane(nrow=2, ncol=2, fov=3, 
                      **beam_opts)

soscan.allocate_maps(nside=256)

scan_opts = dict(                 
                 scan_speed=scan_speed,
                 ctime_func=soscan.schedule_ctime,
                 q_bore_func=soscan.schedule_scan)

chunks = soscan.partition_schedule_file(filename='', 
    chunksize=0.5*soscan.mlen*soscan.fsamp) 

In [ ]:
# Generate timestreams with Gaussian beams
soscan.scan_instrument_mpi(alm, **scan_opts)

maps, cond = soscan.solve_for_map()

In [ ]:
cond[cond == np.inf] = hp.UNSEEN
cart_opts = dict(unit=r'[$\mu K_{\mathrm{CMB}}$]', lonra=[-60, 40], latra=[-70, -40])
hp.mollview(cond, min=2, max=5)
hp.mollview(maps[0], min=-250, max=250)
hp.cartview(maps[1], min=-5, max=5, **cart_opts)
hp.cartview(maps[2], min=-5, max=5, **cart_opts)